<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/multilayer_perceptron/06_workshop_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Dense
from keras.constraints  import maxnorm
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [8]:
data = pd.read_csv('https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_clasification_binary_multiclass/sonar_csv.csv')
data_values = data.values

# Split

In [9]:
X = data_values[:, 0:60].astype('float64')
y = data_values[:,60]

# Encoder

In [10]:
encoder = LabelEncoder()
encoder_y = encoder.fit_transform(y)

In [11]:
def base_model():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, activation='relu', kernel_initializer='glorot_uniform'))
  model.add(Dense(30, activation='relu', kernel_initializer='glorot_uniform'))
  model.add(Dense(1, activation='sigmoid'))
  sgd = SGD(learning_rate=0.01, momentum=0.8)
  model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
  return model

In [12]:
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn=base_model, epochs = 300, batch_size = 16, verbose = 0)))

In [14]:
pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoder_y, cv = kfold)
print(" Modelo de linea base: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Modelo de linea base: 87.45% (4.02%)


# Droput Capa Entrada

In [15]:
# Tasa de droput 
# Se impone una restricción en los pesos para cada capa oculta con la norma maxima de los pesos para que no exceda 3, estableciendo el argumento de kernel_constraint en la clase Dense
# Aumenta la tasa de aprendizaje y el momentum

def input_dropout_model():
  model = Sequential()
  model.add(Dropout(0.2, input_shape = (60,)))
  model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3), kernel_initializer='glorot_uniform'))
  model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3), kernel_initializer='glorot_uniform'))
  model.add(Dense(1, activation='sigmoid'))

  sgd = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
  return model

In [18]:
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn=input_dropout_model, epochs = 300, batch_size = 16, verbose = 0)))

pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline,X, encoder_y, cv = kfold)
print("Modelo de Dropout en la entrada: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo de Dropout en la entrada: 86.07% (4.94%)


In [20]:
def dropout_hidden_model():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, activation='relu', kernel_constraint=maxnorm(1), kernel_initializer='glorot_uniform'))
  model.add(Dropout(0.2))
  model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(1), kernel_initializer='glorot_uniform'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))

  sgd = SGD(learning_rate=0.1, momentum=0.9)
  model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics = ['accuracy'])

  return model

In [21]:
estimator = []
estimator.append(('standarize',StandardScaler()))
estimator.append(('MLP', KerasClassifier(build_fn=dropout_hidden_model, epochs = 300, batch_size=16,verbose=0)))

pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)

results = cross_val_score(pipeline, X, encoder_y, cv=kfold)
print(" Modelo de Dropout en la capa oculta: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Modelo de Dropout en la capa oculta: 83.19% (7.71%)


# Recomendaciones


1.   Utilizar por lo general un valor de Dropout del 20% - 50%
2.   Utilizar drooupt en la entrada(Capa visible) y capas ocultas
3.   Utilizar una gran tasa de aprendizaje (aumentar el factor de 0.01 a 10% o 100%) y un momentum de 0.9 a 0.99
4. Restrinja el tamaño de los pesos de la red. Una gran tasa de aprendizaje puede resultar en pesos de red muy grandes

